In [26]:
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup as bs
import requests as req

# Scrape everything


In [27]:
# this dictionary will hold everything we pull from all the sites
scraped_data = {}

In [28]:
# site 1 -
news_url = "https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest" # probably need to replace this since it redirects
newsResponse = req.get(news_url)

newsSoup = bs(newsResponse.text, "html.parser")
#print(newsSoup.prettify())

newsTitle = newsSoup.find("div", class_="content_title").text
scraped_data['news_title'] = newsTitle

newsPara = newsSoup.find("div", class_="rollover_description_inner").text
scraped_data['news_paragraph'] = newsPara

In [29]:
# site 2 - https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars

executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

fimageWebUrl = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(fimageWebUrl)

html = browser.html
imageSoup = bs(html, "html.parser")

In [30]:
fimageUrlHtml = imageSoup.find("div", class_= "carousel_items").find("article")["style"].split("'")[1]
scraped_data['featured_image_url'] = fimageWebUrl + fimageUrlHtml
print(scraped_data['featured_image_url'])

https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars/spaceimages/images/wallpaper/PIA23354-1920x1200.jpg


In [31]:
# site 3 - https://twitter.com/marswxreport?lang=en

twitterSite = req.get("https://twitter.com/marswxreport?lang=en")
twitterSoup = bs(twitterSite.text, 'html.parser')

tweetSiteInfo = twitterSoup.find_all('div', class_="js-tweet-text-container")
marsWeather = tweetSiteInfo[0].text
print(marsWeather)
scraped_data['Mars_Weather'] = marsWeather

#mars_weather = 'Sol 1801 (Aug 30, 2017), Sunny, high -21C/-5F, low -80C/-112F, pressure at 8.82 hPa, daylight 06:09-17:55'


InSight sol 309 (2019-10-10) low -102.3ºC (-152.1ºF) high -26.2ºC (-15.1ºF)
winds from the SSE at 6.1 m/s (13.6 mph) gusting to 18.9 m/s (42.4 mph)
pressure at 7.20 hPapic.twitter.com/sSOjseIl81



In [32]:
# site 4 - 
marsFactsSite = "https://space-facts.com/mars/"
marsFacts = pd.read_html(marsFactsSite)
marsFacts_df = marsFacts[0]

marsFactsHtml = marsFacts_df.to_html(header = False, index = False)
scraped_data['Mars_Facts'] = marsFactsHtml
print(marsFactsHtml)

<table border="1" class="dataframe">
  <tbody>
    <tr>
      <td>Diameter:</td>
      <td>6,779 km</td>
      <td>12,742 km</td>
    </tr>
    <tr>
      <td>Mass:</td>
      <td>6.39 × 10^23 kg</td>
      <td>5.97 × 10^24 kg</td>
    </tr>
    <tr>
      <td>Moons:</td>
      <td>2</td>
      <td>1</td>
    </tr>
    <tr>
      <td>Distance from Sun:</td>
      <td>227,943,824 km</td>
      <td>149,598,262 km</td>
    </tr>
    <tr>
      <td>Length of Year:</td>
      <td>687 Earth days</td>
      <td>365.24 days</td>
    </tr>
    <tr>
      <td>Temperature:</td>
      <td>-153 to 20 °C</td>
      <td>-88 to 58°C</td>
    </tr>
  </tbody>
</table>


In [36]:
# site 5 
# use bs4 to scrape the title and url and add to dictionary
# Example:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

articleSite = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(articleSite)

HemsHtml = browser.html
soup = bs(HemsHtml, 'html.parser')

In [37]:

marsHems=[]

for i in range (4):
    HemsImages = browser.find_by_tag('h3')
    HemsImages[i].click()
    html = browser.html
    hemSoup = bs(html, 'html.parser')
    partialHems = hemSoup.find("img", class_="wide-image")["src"]
    imgTitleHems = hemSoup.find("h2",class_="title").text
    imgUrlHems = 'https://astrogeology.usgs.gov'+ partialHems
    dictionaryHems={"title":imgTitleHems,"img_url":imgUrlHems}
    marsHems.append(dictionaryHems)
    
    browser.back()

scraped_data['Mars_Hem_Images'] = marsHems    
print(marsHems)

[{'title': 'Cerberus Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg'}, {'title': 'Schiaparelli Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg'}, {'title': 'Syrtis Major Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg'}, {'title': 'Valles Marineris Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg'}]
{'news_title': '\n\nNASA Invites Students to Name Mars 2020 Rover\n\n', 'news_paragraph': "\nThrough Nov. 1, K-12 students in the U.S. are encouraged to enter an essay contest to name NASA's next Mars rover.\n", 'featured_image_url': 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars/spaceimages/imag

In [ ]:
# File-> download as python into a new module called scrape_mars.py

In [ ]:
# use day 3 09-Ins_Scrape_And_Render/app.py as a blue print on how to finish the homework.

# replace the contents of def index() and def scraper() appropriately.

# change the index.html to render the site with all the data.